<a href="https://colab.research.google.com/github/som1966/Final-Project/blob/zohairBranch/finalProject_firstsegment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

reading in files from data storage so that we can manipulate them

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubun

First, enter the following code to download a Postgres driver that will allow Spark to interact with Postgres:

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar


--2022-01-17 08:18:58--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.6’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-01-17 08:18:58 (6.49 MB/s) - ‘postgresql-42.2.16.jar.6’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinalProject-ETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

performed the  first two steps of the ETL process before pyspark

In [ ]:
# connect to data storage --- Extract 
# Read in data from S3 Buckets
from pyspark import SparkFiles
url1 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/Happiness_Scores_2019.csv"
spark.sparkContext.addFile(url1)
Happiness_Scores_2019_df = spark.read.csv(SparkFiles.get("Happiness_Scores_2019.csv"), sep=",", header=True, inferSchema=True)
Happiness_Scores_2019_df = Happiness_Scores_2019_df.drop('yr') \
       .drop('country_name')

Happiness_Scores_2019_df.show()

+------------+-----------+
|country_code|Life Ladder|
+------------+-----------+
|           4|      2.375|
|           8|      4.995|
|          12|      4.745|
|          32|      6.086|
|          51|      5.488|
|          36|      7.234|
|          40|      7.195|
|          31|      5.173|
|          48|      7.098|
|          50|      5.114|
|         112|      5.821|
|          56|      6.772|
|         204|      4.976|
|          68|      5.674|
|          70|      6.016|
|          72|      3.471|
|          76|      6.451|
|         100|      5.108|
|         854|      4.741|
|         116|      4.998|
+------------+-----------+
only showing top 20 rows



In [ ]:
from pyspark import SparkFiles
url2 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_Consumer_Price+Index_2019.csv"
spark.sparkContext.addFile(url2)
UN_Consumer_Price_Index_2019_df = spark.read.csv(SparkFiles.get("UN_Consumer_Price+Index_2019.csv"), sep=",", header=True, inferSchema=True)
UN_Consumer_Price_Index_2019_df.show()

+-------------------+-------------------+----+--------------------+-----+
|Region/Country/Area|                _c1|Year|              Series|Value|
+-------------------+-------------------+----+--------------------+-----+
|                  4|        Afghanistan|2019|Consumer price in...|149.9|
|                  8|            Albania|2019|Consumer price in...|119.1|
|                 12|            Algeria|2019|Consumer price in...|151.4|
|                 20|            Andorra|2019|Consumer price in...|105.8|
|                 24|             Angola|2019|Consumer price in...|378.9|
|                660|           Anguilla|2019|Consumer price in...|107.1|
|                 28|Antigua and Barbuda|2019|Consumer price in...|115.4|
|                 32|          Argentina|2019|Consumer price in...|232.8|
|                 51|            Armenia|2019|Consumer price in...|129.2|
|                533|              Aruba|2019|Consumer price in...|109.2|
|                 36|          Austral

In [ ]:
url3 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_Country.csv"
spark.sparkContext.addFile(url3)
UN_Country_df = spark.read.csv(SparkFiles.get("UN_Country.csv"), sep=",", header=True, inferSchema=True)
UN_Country_df.show()

+------------+-------------------+
|country_code|       country_name|
+------------+-------------------+
|           4|        Afghanistan|
|           8|            Albania|
|          12|            Algeria|
|          16|     American Samoa|
|          20|            Andorra|
|          24|             Angola|
|         660|           Anguilla|
|          28|Antigua and Barbuda|
|          32|          Argentina|
|          51|            Armenia|
|         533|              Aruba|
|          36|          Australia|
|          40|            Austria|
|          31|         Azerbaijan|
|          44|            Bahamas|
|          48|            Bahrain|
|          50|         Bangladesh|
|          52|           Barbados|
|         112|            Belarus|
|          56|            Belgium|
+------------+-------------------+
only showing top 20 rows



In [ ]:
url4 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_GDP_Per_Capita_2019.csv"
spark.sparkContext.addFile(url4)
UN_GDP_Per_Capita_2019_df = spark.read.csv(SparkFiles.get("UN_GDP_Per_Capita_2019.csv"), sep=",", header=True, inferSchema=True)
UN_GDP_Per_Capita_2019_df.show()

+-------------------+-------------------+----+--------------------+----------+
|Region/Country/Area|                _c1|Year|              Series|     Value|
+-------------------+-------------------+----+--------------------+----------+
|                  4|        Afghanistan|2019|GDP per capita (U...|   $470.00|
|                  8|            Albania|2019|GDP per capita (U...| $5,303.00|
|                 12|            Algeria|2019|GDP per capita (U...| $3,976.00|
|                 20|            Andorra|2019|GDP per capita (U...|$40,887.00|
|                 24|             Angola|2019|GDP per capita (U...| $2,671.00|
|                660|           Anguilla|2019|GDP per capita (U...|$25,529.00|
|                 28|Antigua and Barbuda|2019|GDP per capita (U...|$17,113.00|
|                 32|          Argentina|2019|GDP per capita (U...|$10,041.00|
|                 51|            Armenia|2019|GDP per capita (U...| $4,623.00|
|                533|              Aruba|2019|GDP pe

In [ ]:
url5 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_Gender_Ratio_2019.csv"
spark.sparkContext.addFile(url5)
UN_Gender_Ratio_2019_df = spark.read.csv(SparkFiles.get("UN_Gender_Ratio_2019.csv"), sep=",", header=True, inferSchema=True)
UN_Gender_Ratio_2019_df.show()

+-------------------+-------------------+----+--------------------+-----+
|Region/Country/Area|                _c1|Year|              Series|Value|
+-------------------+-------------------+----+--------------------+-----+
|                  4|        Afghanistan|2019|Sex ratio (males ...|105.5|
|                  8|            Albania|2019|Sex ratio (males ...|103.7|
|                 12|            Algeria|2019|Sex ratio (males ...|102.1|
|                 24|             Angola|2019|Sex ratio (males ...| 97.9|
|                 28|Antigua and Barbuda|2019|Sex ratio (males ...| 93.2|
|                 32|          Argentina|2019|Sex ratio (males ...| 95.2|
|                 51|            Armenia|2019|Sex ratio (males ...| 88.8|
|                533|              Aruba|2019|Sex ratio (males ...| 90.3|
|                 36|          Australia|2019|Sex ratio (males ...| 99.2|
|                 40|            Austria|2019|Sex ratio (males ...| 97.0|
|                 31|         Azerbaij

In [ ]:
url6 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_Infant_Mortality_2020.csv"
spark.sparkContext.addFile(url6)
UN_Infant_Mortality_2020_df = spark.read.csv(SparkFiles.get("UN_Infant_Mortality_2020.csv"), sep=",", header=True, inferSchema=True)

#Remove multiple columnss in one go
UN_Infant_Mortality_2020_df = UN_Infant_Mortality_2020_df.drop('Year')
UN_Infant_Mortality_2020_df = UN_Infant_Mortality_2020_df.drop('_c1')\
        .drop('series')
UN_Infant_Mortality_2020_df.withColumnRenamed("Value","infant_mortality_per1000_births")
UN_Infant_Mortality_2020_df.show(truncate=False)

+-------------------+-----+
|Region/Country/Area|Value|
+-------------------+-----+
|4                  |51.7 |
|8                  |8.0  |
|12                 |21.2 |
|24                 |61.5 |
|28                 |5.2  |
|32                 |10.2 |
|51                 |10.8 |
|533                |13.6 |
|36                 |3.1  |
|40                 |3.2  |
|31                 |20.8 |
|44                 |5.9  |
|48                 |6.0  |
|50                 |26.8 |
|52                 |10.0 |
|112                |3.0  |
|56                 |2.8  |
|84                 |12.8 |
|204                |61.1 |
|64                 |24.1 |
+-------------------+-----+
only showing top 20 rows



In [ ]:

url7 ="https://corti-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_Life_Expectancy_2020.csv"
spark.sparkContext.addFile(url7)
UN_Life_Expectancy_2020_df = spark.read.csv(SparkFiles.get("UN_Life_Expectancy_2020.csv"), sep=",", header=True, inferSchema=True)
UN_Life_Expectancy_2020_df = UN_Life_Expectancy_2020_df.drop('Year')
UN_Life_Expectancy_2020_df = UN_Life_Expectancy_2020_df.drop('_c1')\
        .drop('series')
UN_Life_Expectancy_2020_df.withColumnRenamed("Value","Life_Expectancy")

UN_Life_Expectancy_2020_df.show()

Py4JJavaError: ignored

In [ ]:
url ="https://zohair-gw-group-project.s3.amazonaws.com/Unprocessed+Data/UN_Population_Density_2019.csv"
spark.sparkContext.addFile(url)
table8_df = spark.read.csv(SparkFiles.get("UN_Population_Density_2019.csv"), sep=",", header=True, inferSchema=True)
table8_df.show()

Py4JJavaError: ignored

In [ ]:
# Transform

UN_Infant_Mortality_2020_df.printSchema()

https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/

https://sparkbyexamples.com/pyspark/pyspark-rename-dataframe-column/


https://jacobcelestine.com/knowledge_repo/colab_and_pyspark/



In [ ]:
UN_Infant_Mortality_2020_df.withColumnRenamed("Value","infant_mortality_per1000_births").printSchema()


In [ ]:
#Remove multiple columnss in one go
UN_Infant_Mortality_2020_df = UN_Infant_Mortality_2020_df.drop('Year')
UN_Infant_Mortality_2020_df.show

In [ ]:
UN_Infant_Mortality_2020_df = UN_Infant_Mortality_2020_df.drop('_c1')\
        .drop('series')
UN_Infant_Mortality_2020_df.show

In [ ]:
Happiness_Scores_2019_df = Happiness_Scores_2019_df.drop('yr') \
       .drop('country_name')
Happiness_Scores_2019_df.show

In [ ]:
Happiness_Scores_2019_df.show()